In [20]:
"""
Project: Interactive Customer Segmentation Dashboard

Objective:
- Segment customers based on demographics and spending behavior
- Identify meaningful customer groups
- Provide insights for business decision-making
- Visualize segments using Tableau

Tools:
Python, Pandas, Scikit-learn, Tableau
"""


'\nProject: Interactive Customer Segmentation Dashboard\n\nObjective:\n- Segment customers based on demographics and spending behavior\n- Identify meaningful customer groups\n- Provide insights for business decision-making\n- Visualize segments using Tableau\n\nTools:\nPython, Pandas, Scikit-learn, Tableau\n'

In [21]:
# Import required libraries and load dataset
import pandas as pd

df = pd.read_csv("customers-1000.csv")
df.head()


,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,dE014d010c7ab0c,Andrew,Goodman,Stewart-Flynn,Rowlandberg,Macao,846-790-4623x4715,(422)787-2331x71127,marieyates@gomez-spencer.info,2021-07-26,http://www.shea.biz/
1,2,2B54172c8b65eC3,Alvin,Lane,"Terry, Proctor and Lawrence",Bethside,Papua New Guinea,124-597-8652x05682,321.441.0588x6218,alexandra86@mccoy.com,2021-06-24,http://www.pena-cole.com/
2,3,d794Dd48988d2ac,Jenna,Harding,Bailey Group,Moniquemouth,China,(335)987-3085x3780,001-680-204-8312,justincurtis@pierce.org,2020-04-05,http://www.booth-reese.biz/
3,4,3b3Aa4aCc68f3Be,Fernando,Ford,Moss-Maxwell,Leeborough,Macao,(047)752-3122,048.779.5035x9122,adeleon@hubbard.org,2020-11-29,http://www.hebert.com/
4,5,D60df62ad2ae41E,Kara,Woods,Mccarthy-Kelley,Port Jacksonland,Nepal,+1-360-693-4419x19272,163-627-2565,jesus90@roberson.info,2022-04-22,http://merritt.com/


In [22]:
df.shape


(1000, 12)

In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Index              1000 non-null   int64 
 1   Customer Id        1000 non-null   object
 2   First Name         1000 non-null   object
 3   Last Name          1000 non-null   object
 4   Company            1000 non-null   object
 5   City               1000 non-null   object
 6   Country            1000 non-null   object
 7   Phone 1            1000 non-null   object
 8   Phone 2            1000 non-null   object
 9   Email              1000 non-null   object
 10  Subscription Date  1000 non-null   object
 11  Website            1000 non-null   object
dtypes: int64(1), object(11)
memory usage: 93.9+ KB


In [24]:
df.describe()


,Index
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


In [25]:
df.columns


Index(['Index', 'Customer Id', 'First Name', 'Last Name', 'Company', 'City',
       'Country', 'Phone 1', 'Phone 2', 'Email', 'Subscription Date',
       'Website'],
      dtype='object')

In [26]:
df.columns = df.columns.str.strip().str.replace(" ", "_")
df.columns


Index(['Index', 'Customer_Id', 'First_Name', 'Last_Name', 'Company', 'City',
       'Country', 'Phone_1', 'Phone_2', 'Email', 'Subscription_Date',
       'Website'],
      dtype='object')

In [27]:
df['Subscription_Date'] = pd.to_datetime(df['Subscription_Date'])
df['Subscription_Date'].head()


0   2021-07-26
1   2021-06-24
2   2020-04-05
3   2020-11-29
4   2022-04-22
Name: Subscription_Date, dtype: datetime64[ns]

In [28]:
today = pd.Timestamp('today')
df['Tenure_Days'] = (today - df['Subscription_Date']).dt.days

df[['Customer_Id', 'Tenure_Days']].head()


,Customer_Id,Tenure_Days
0,dE014d010c7ab0c,1616
1,2B54172c8b65eC3,1648
2,d794Dd48988d2ac,2093
3,3b3Aa4aCc68f3Be,1855
4,D60df62ad2ae41E,1346


In [29]:
df['Is_Company_Customer'] = df['Company'].notnull().astype(int)
df[['Company', 'Is_Company_Customer']].head()


,Company,Is_Company_Customer
0,Stewart-Flynn,1
1,"Terry, Proctor and Lawrence",1
2,Bailey Group,1
3,Moss-Maxwell,1
4,Mccarthy-Kelley,1


In [30]:
df['Email_Domain'] = df['Email'].str.split('@').str[1]
df[['Email', 'Email_Domain']].head()


,Email,Email_Domain
0,marieyates@gomez-spencer.info,gomez-spencer.info
1,alexandra86@mccoy.com,mccoy.com
2,justincurtis@pierce.org,pierce.org
3,adeleon@hubbard.org,hubbard.org
4,jesus90@roberson.info,roberson.info


In [31]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Country_Encoded'] = le.fit_transform(df['Country'])
df['Email_Domain_Encoded'] = le.fit_transform(df['Email_Domain'])


In [32]:
features = df[
    ['Tenure_Days', 'Is_Company_Customer', 'Country_Encoded', 'Email_Domain_Encoded']
]

features.head()


,Tenure_Days,Is_Company_Customer,Country_Encoded,Email_Domain_Encoded
0,1616,1,124,335
1,1648,1,166,549
2,2093,1,44,669
3,1855,1,124,425
4,1346,1,149,729


In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [34]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)
df['Segment'] = kmeans.fit_predict(scaled_features)

df[['Customer_Id', 'Segment']].head()


,Customer_Id,Segment
0,dE014d010c7ab0c,0
1,2B54172c8b65eC3,3
2,d794Dd48988d2ac,2
3,3b3Aa4aCc68f3Be,2
4,D60df62ad2ae41E,3


In [35]:
df.groupby('Segment').mean(numeric_only=True)


,Index,Tenure_Days,Is_Company_Customer,Country_Encoded,Email_Domain_Encoded
Segment,,,,,
0,501.360360,1738.130631,1.0,183.680180,226.243243
1,517.301205,1793.851406,1.0,51.995984,239.979920
2,481.216102,2012.156780,1.0,140.046610,671.745763
3,501.102389,1521.034130,1.0,107.890785,663.378840
